In [ ]:
import gensim
import numpy as np
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_20newsgroups
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from sklearn import preprocessing
import nltk
import re
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [ ]:
class Classify20:
    def myTokenizer(self,text):
        return nltk.regexp_tokenize(text, "\\b[a-zA-Z]{3,}\\b")
    def tokenizeContents(self,contents):
        li=re.findall(self.regex,contents)
        return [self.myTokenizer(content) for content in li]
    def preprocess(self):
        self.model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Arfat/Downloads/Compressed/GoogleNews-vectors-negative300.bin', binary=True)
        self.twenty_all = fetch_20newsgroups(subset='all', shuffle=True,remove=('headers', 'footers', 'quotes'))
        full1=[" ".join(data.split("\n")) for data in self.twenty_all.data]
        stopWords = set(stopwords.words('english'))
        self.full= []
        tmp=[] 
        for d in range(len(full1)):
            words = word_tokenize(full1[d])
            for w in words:
                if w not in stopWords:
                    tmp.append(w)
            x=" ".join(tmp)
            tmp=self.myTokenizer(x)
            x=" ".join(tmp)
            tmp=[]
            self.full.append(x)
        self.skf = StratifiedKFold(n_splits=3,shuffle=True,random_state=1)
        y=self.twenty_all.target
        self.regex = r'\b\w+\b'
        self.readydocs()
    def readydocs(self):
        add=np.zeros(300,dtype='float32')
        self.doc=np.zeros((18846,300))
        self.l=[]#for empty documents
        count=0 #total times the key error is raised
        c_names=[] #contains words that are in documents but they don't have vector representations
        count_word_exists=0 #checks number of words having vector representations in the current document
        self.del_index=[] # has index of the documnets where no word has a vector representation

        rate=[] #hit rate for words having vector representations in a document
        for q1 in range(len(self.full)):
            token=self.tokenizeContents(self.full[q1])
            token = list(filter(None, token))
            if len(token)==0:
                self.l.append(q1)
                rate.append(0)
                self.doc[q1]=np.zeros(300,dtype='float32')
                continue
            for q in range(len(token)):
                try:
                    add=add+self.model.word_vec("".join(token[q]))
                    count_word_exists=count_word_exists+1

                except KeyError:
                    count+=1
                    if "".join(token[q]) not in c_names:
                        c_names.append("".join(token[q]))
                    continue
        #print(add,len(token))
            if count_word_exists==0:
                self.del_index.append(q1)
                add=np.zeros(300,dtype='float32')
                self.doc[q1]=add
                rate.append(0)
                continue
            rate.append(count_word_exists/len(token))
            add=add/count_word_exists
            self.doc[q1]=add
            add=np.zeros(300,dtype='float32')
            count_word_exists=0
        self.doc=[ value for (i, value) in enumerate(self.doc) if i not in set(self.l+self.del_index) ]
        self.doc=np.asarray(self.doc)
        self.delundocs()###delete unnecessary docs
    def delundocs(self):
        self.y=self.twenty_all.target                                 ##############deleting docs for all methods
        self.full=[ value for (i, value) in enumerate(self.full) if i not in set(self.l+self.del_index) ]
        for i in sorted(set(self.l+self.del_index),reverse=True):
            self.y=np.delete(self.y,i)
    def w2v(self):
    ##########W2V
        clf=LinearSVC()
        for k,(train_index, test_index) in enumerate(self.skf.split(self.doc, self.y)):
            clf.fit(self.doc[train_index],self.y[train_index])
            y_pred=clf.predict(self.doc[test_index])
            print("Fold :",k)
            print(classification_report(y_pred,self.y[test_index],target_names=self.twenty_all.target_names))
    def tfidf(self):
    ##################              TF-IDF
        tfvec=TfidfVectorizer(ngram_range=(1,2))
        X=tfvec.fit_transform(self.full)
        clf = MultinomialNB()
        for k,(train_index, test_index) in enumerate(self.skf.split(X, self.y)):

            clf.fit(X[train_index],self.y[train_index])
            y_pred=clf.predict(X[test_index])
            print("Fold :",k)
            print(classification_report(y_pred,self.y[test_index],target_names=self.twenty_all.target_names))
    def ftext(self,cmd):
    ##################              Fast text
        cmd1="C:/Users/Arfat/Downloads/Compressed/amazonreviews/fasttext supervised "
        cmd1=cmd1+cmd
        cmd1=cmd1+" -input train20.txt -output model20 >> ftrainresult.txt"
        #print (cmd1)
        #return
        cmd2="C:/Users/Arfat/Downloads/Compressed/amazonreviews/fasttext test model20.bin test20.txt >> ftestresult.txt"
        cmd3="C:/Users/Arfat/Downloads/Compressed/amazonreviews/fasttext predict model20.bin test20.txt > result20.txt"
        c=0
        for k,(train_index, test_index) in enumerate(self.skf.split(self.full, self.y)):
            outF = open("train20.txt", "w")
            out1 = open("test20.txt", "w")
            for i in range(len(train_index)):
                outF.write("__label__")
                outF.write(str(self.y[train_index[i]]))
                outF.write(" ")
                try :
                    outF.write(self.full[train_index[i]])
                    outF.write("\n")
                except UnicodeEncodeError:
                    print("i",i)
                    outF.write("\n")
                    continue
            os.system(cmd1)
            print(len(test_index))
            c=0
            for j in range(len(test_index)):
                out1.write("__label__")
                out1.write(str(self.y[test_index[j]]))
                out1.write(" ")
                c=c+1
                try :
                    out1.write(self.full[test_index[j]])
                    out1.write("\n")
                except UnicodeEncodeError:
                    print("j",j)
                    out1.write("\n")
                    continue
            outF.close()
            out1.close()
            os.system(cmd2)
            os.system(cmd3)
            f = open('result20.txt', 'r')
            lines = np.asarray([np.int32("".join(re.findall('\d+', line))) for line in f.readlines()])
            f.close()
            print(classification_report(lines,self.y[test_index],target_names=self.twenty_all.target_names))


    
    

In [ ]:
c20=Classify20()

In [ ]:
c20.preprocess()

In [ ]:
c20.tfidf()

In [ ]:
c20.w2v()


In [ ]:
c20.ftext('-dim 100 -epoch 100')